## Testing ##

In [1]:
import lxml.etree as etree
import re
from datetime import datetime

In [4]:
tree = etree.parse('House_of_Representatives_2011_05_10_10_Official.xml')
etree.strip_tags(tree, 'inline')
root = tree.getroot()

In [7]:
# defining divs
divs =  tree.findall("//debate")

In [5]:
for child in root: 
    print(child.tag)

session.header
chamber.xscript
answers.to.questions


In [ ]:
# access session info and store in a list 
for element in root.iter('session.header'):
    for child in element:
        print("%s - %s" % (child.tag, child.text))
        
for element in root.iter('session.header'):
    for child in element.iter('date'):
        print("%s - %s" % (child.tag, child.text))

In [6]:
for i in root.xpath('//talk.start'):
    j = i.getparent().tag
    # print(j)

### Names ###

In [ ]:
# access speaker names, only metadata and not display names 
# if possible, get name.id, electorate, party and role as well
# this works 
for element in root.iter('talker'):
    for child in element.iter('name'):
        if child.get('role') == 'metadata':
            print("%s - %s" %(child.tag, child.text)) 

# further issues: how to get name of The Speaker
# access from a list? 

In [15]:
namemd = []
for div in divs:
    for uts in div.iter('talk.start'):
        for name in uts.xpath('talker//name'):
            if name.get('role') == 'metadata':
                namemd = name.text.split(', ')
                if len(namemd) == 1:
                    names = namemd[0]
                else:
                    names = namemd[1] + ' ' + namemd[0]
                #cleaning text
                names = re.sub(r'\([^)]*\)', '', names)
                names = names.replace('  ', ' ')
                names = names.replace('Dr ', '')
                names = names.replace('Mr ', '')
                #print(names)

### Speaker Metadata ###

In [17]:
#Dictionary
for div in divs:
    for uts in div.iter('talk.start'):
        info_dict = {}
        info_dict['nameid'] = uts.xpath('talker/name.id/text()')
        info_dict['party'] = uts.xpath('talker/party/text()')
        info_dict['electorate'] = uts.xpath('talker/electorate/text()')
        info_dict['role'] = uts.xpath('talker/role/text()')

In [23]:
#Name ID
for div in divs:
    for uts in div.iter('talk.start'):
        for nameidxp in uts.xpath('talker/name.id/text()'): info_dict['nameid'] = nameidxp
        #print(type(info_dict['nameid']))
        #print(info_dict['nameid'])

In [18]:
#Parties 
for div in divs:
    for uts in div.iter('talk.start'):
        info_dict['party'] = uts.xpath('talker/party/text()')
        #for elem in info_dict['party']:
            #print(elem)

In [9]:
#Special Roles
roles = root.xpath('//role')
for role in roles:
    print(role.text)

Prime Minister
Minister for Health and Aged Care
Prime Minister
Prime Minister
Treasurer
Prime Minister
Minister for Employment, Workplace Relations and Small Business
Prime Minister
Treasurer
Prime Minister
Deputy Prime Minister
Prime Minister
Minister for Finance and Administration
Prime Minister
Minister for Trade
Prime Minister
Minister for Health and Aged Care
Prime Minister
Minister for Education, Training and Youth Affairs and Minister Assisting the Prime Minister for the Public Service
Prime Minister
Leader of the Opposition
Prime Minister
Treasurer
Minister for Health and Aged Care
Minister for Employment, Workplace Relations and Small Business
Attorney-General
Parliamentary Secretary to the Minister for Finance and Administration
Parliamentary Secretary to the Minister for Finance and Administration
Parliamentary Secretary to the Minister for Employment, Workplace Relations and Small Business
Minister for Agriculture, Fisheries and Forestry


### Texts ### 

In [8]:
# prints all para inside talk.start 
for div in divs:
    for uts in div.iter('talk.start'):
        text = []
        for i in uts.iter():
            if i.tag=="p":
                if i.text:
                    text.append(i.text.strip())
                    print(text)

In [9]:
# prints all para outside talk.start 
for div in divs:
    for uts in div.iter('talk.start'):
        for i in uts.xpath('following-sibling::p'):
            text = []
            if i.text:
                text.append(i.text.strip())
                print(text)

In [ ]:
# combination? 
for div in divs: 
    #if div.tag == "talk.start": # necessary?
    for uts in div.iter('talk.start'):
        # text inside talk.start
        for i in uts.iter():
            text = []
            if i.tag == "para":
                if i.text:
                    text.append(i.text.strip())
                    print(text)
        # text outside talk.start
        for i in uts.xpath('following-sibling::para'):
            text2 = [] # has to be here otherwise will be repeated, but deletes previous
            if i.text:
                text2.append(i.text.strip())
                print(text)
    #else:

In [5]:
# combination v2? 
for div in divs:
    for uts in div.iter('talk.start'):  
        print("#######")
        for i in uts.iter():
            text = []
            if i.tag=="para":
                if i.text:
                    text.append(i.text.strip(u'\u2014'))
                    print(text)
                    
        for j in uts.xpath('following-sibling::para|parent::*/following-sibling::para|following-sibling::*/child::para'):
            text2 = []
            if j.text:
                text2.append(j.text.strip(u'\u2014'))
                print(text2)

#######
['In accordance with standing order 41(h), and the recommendations of the whips adopted by the House on 11 March 2009, I present copies of the terms of motions for which notice has been given by the members for Mallee, Lyons, Mayo and Dobell. These matters will be considered by the Main Committee later today.']
#######
['I have received advice from the Chief Government Whip that he has nominated Mr Murphy to be a member of the Standing Committee on Climate Change, Water, Environment and the Arts in place of Mrs D’Ath.']
#######
['In speaking on the Commonwealth Electoral Amendment (Political Donations and Other Measures) Bill 2009 this morning, there are several general comments that I wish to begin with. The coalition strongly support comprehensive reform of the Commonwealth Electoral Act and, in our time in government, introduced many reforms to the Commonwealth Electoral Act which improved its operation and which tightened up issues to do with transparency, accountability an

['Other members opposite, such as the member for Wills and the member for Melbourne concurred, stating that they would not be increasing the financial burden of our students. This bill does the complete opposite. It seeks to introduce a compulsory payment for non-academic services through the imposition of a deferred fee. Imposing an extra burden on the students at a time of economic uncertainty is surely not in the best interests of Australian students.']
['In conclusion, I say that the purpose of this bill is simply to introduce compulsory student unionism by stealth, by denying students their fundamental right to freedom of choice on campus. As I have said before, it is the students—not the government, universities nor student unions—who are in the best position to determine their financial priorities. This bill will deny students a real choice on campus and will force them to fund activities which they may not need or want. I will return to Hal Colebatch’s article. In his last para